# Linear Regression
:label:`sec_linear_regression`

*Regression* problems pop up whenever we want to predict a numerical value.
Common examples include predicting prices (of homes, stocks, etc.),
predicting the length of stay (for patients in the hospital),
forecasting demand (for retail sales), among numerous others.
Not every prediction problem is one of classical regression.
Later on, we will introduce classification problems,
where the goal is to predict membership among a set of categories.

As a running example, suppose that we wish
to estimate the prices of houses (in dollars)
based on their area (in square feet) and age (in years).
To develop a model for predicting house prices,
we need to get our hands on data,
including the sales price, area, and age for each home.
In the terminology of machine learning,
the dataset is called a *training dataset* or *training set*,
and each row (containing the data corresponding to one sale)
is called an *example* (or *data point*, *instance*, *sample*).
The thing we are trying to predict (price)
is called a *label* (or *target*).
The variables (age and area)
upon which the predictions are based
are called *features* (or *covariates*).

# 선형 회귀
:label:`sec_linear_regression`

*회귀* 문제는 수치 값을 예측할 때마다 발생합니다.일반적인 예로는 주택, 주식 등의 가격 예측,
병원 환자의 입원 기간 예측,
소매 판매 수요 예측 등이 있습니다.모든 예측 문제가 고전적인 회귀 분석의 문제는 아닙니다.나중에 분류 문제를 소개하겠습니다. 이 문제의 목표는
범주 집합에 속하는지 여부를 예측하는 것입니다.실행 예제로, 주택의 면적(제곱피트)과 연식(년)을 기반으로 주택 가격(달러)을 추정한다고 가정해 보겠습니다.주택 가격 예측 모델을 개발하려면
각 주택의 매매 가격, 면적, 연식을 포함한 데이터를 확보해야 합니다.머신 러닝 용어로
데이터셋을 *훈련 데이터셋* 또는 *훈련 세트*라고 하며,
각 행(한 판매에 해당하는 데이터를 포함)을
*예시*(또는 *데이터 포인트*, *인스턴스*, *샘플*)라고 합니다.예측하려는 대상(가격)을
*레이블*(또는 *타겟*)이라고 합니다.예측의 기반이 되는 변수(연령 및 지역)를
*특성*(또는 *공변량*)이라고 합니다.


In [1]:
#%matplotlib inline
import math
import time
import numpy as np
import torch
from d2l import torch as d2l

## Basics

*Linear regression* is both the simplest
and most popular among the standard tools
for tackling regression problems.
Dating back to the dawn of the 19th century :cite:`Legendre.1805,Gauss.1809`,
linear regression flows from a few simple assumptions.
First, we assume that the relationship
between features $\mathbf{x}$ and target $y$
is approximately linear,
i.e., that the conditional mean $E[Y \mid X=\mathbf{x}]$
can be expressed as a weighted sum
of the features $\mathbf{x}$.
This setup allows that the target value
may still deviate from its expected value
on account of observation noise.
Next, we can impose the assumption that any such noise
is well behaved, following a Gaussian distribution.
Typically, we will use $n$ to denote
the number of examples in our dataset.
We use superscripts to enumerate samples and targets,
and subscripts to index coordinates.
More concretely,
$\mathbf{x}^{(i)}$ denotes the $i^{\textrm{th}}$ sample
and $x_j^{(i)}$ denotes its $j^{\textrm{th}}$ coordinate.

### Model
:label:`subsec_linear_model`

At the heart of every solution is a model
that describes how features can be transformed
into an estimate of the target.
The assumption of linearity means that
the expected value of the target (price) can be expressed
as a weighted sum of the features (area and age):

$$\textrm{price} = w_{\textrm{area}} \cdot \textrm{area} + w_{\textrm{age}} \cdot \textrm{age} + b.$$
:eqlabel:`eq_price-area`

Here $w_{\textrm{area}}$ and $w_{\textrm{age}}$
are called *weights*, and $b$ is called a *bias*
(or *offset* or *intercept*).
The weights determine the influence of each feature on our prediction.
The bias determines the value of the estimate when all features are zero.
Even though we will never see any newly-built homes with precisely zero area,
we still need the bias because it allows us
to express all linear functions of our features
(rather than restricting us to lines that pass through the origin).
Strictly speaking, :eqref:`eq_price-area` is an *affine transformation* of input features, which is characterized by a *linear transformation* of features via a weighted sum, combined with a *translation* via the added bias.
Given a dataset, our goal is to choose
the weights $\mathbf{w}$ and the bias $b$
that, on average, make our model's predictions
fit the true prices observed in the data as closely as possible.


In disciplines where it is common to focus
on datasets with just a few features,
explicitly expressing models long-form,
as in :eqref:`eq_price-area`, is common.
In machine learning, we usually work
with high-dimensional datasets,
where it is more convenient to employ
compact linear algebra notation.
When our inputs consist of $d$ features,
we can assign each an index (between $1$ and $d$)
and express our prediction $\hat{y}$
(in general the "hat" symbol denotes an estimate) as

$$\hat{y} = w_1  x_1 + \cdots + w_d  x_d + b.$$

Collecting all features into a vector $\mathbf{x} \in \mathbb{R}^d$
and all weights into a vector $\mathbf{w} \in \mathbb{R}^d$,
we can express our model compactly via the dot product
between $\mathbf{w}$ and $\mathbf{x}$:

$$\hat{y} = \mathbf{w}^\top \mathbf{x} + b.$$
:eqlabel:`eq_linreg-y`

In :eqref:`eq_linreg-y`, the vector $\mathbf{x}$
corresponds to the features of a single example.
We will often find it convenient
to refer to features of our entire dataset of $n$ examples
via the *design matrix* $\mathbf{X} \in \mathbb{R}^{n \times d}$.
Here, $\mathbf{X}$ contains one row for every example
and one column for every feature.
For a collection of features $\mathbf{X}$,
the predictions $\hat{\mathbf{y}} \in \mathbb{R}^n$
can be expressed via the matrix--vector product:

$${\hat{\mathbf{y}}} = \mathbf{X} \mathbf{w} + b,$$
:eqlabel:`eq_linreg-y-vec`

where broadcasting (:numref:`subsec_broadcasting`) is applied during the summation.
Given features of a training dataset $\mathbf{X}$
and corresponding (known) labels $\mathbf{y}$,
the goal of linear regression is to find
the weight vector $\mathbf{w}$ and the bias term $b$
such that, given features of a new data example
sampled from the same distribution as $\mathbf{X}$,
the new example's label will (in expectation)
be predicted with the smallest error.

Even if we believe that the best model for
predicting $y$ given $\mathbf{x}$ is linear,
we would not expect to find a real-world dataset of $n$ examples where
$y^{(i)}$ exactly equals $\mathbf{w}^\top \mathbf{x}^{(i)}+b$
for all $1 \leq i \leq n$.
For example, whatever instruments we use to observe
the features $\mathbf{X}$ and labels $\mathbf{y}$, there might be a small amount of measurement error.
Thus, even when we are confident
that the underlying relationship is linear,
we will incorporate a noise term to account for such errors.

Before we can go about searching for the best *parameters*
(or *model parameters*) $\mathbf{w}$ and $b$,
we will need two more things:
(i) a measure of the quality of some given model;
and (ii) a procedure for updating the model to improve its quality.

### Loss Function
:label:`subsec_linear-regression-loss-function`

Naturally, fitting our model to the data requires
that we agree on some measure of *fitness*
(or, equivalently, of *unfitness*).
*Loss functions* quantify the distance
between the *real* and *predicted* values of the target.
The loss will usually be a nonnegative number
where smaller values are better
and perfect predictions incur a loss of 0.
For regression problems, the most common loss function is the squared error.
When our prediction for an example $i$ is $\hat{y}^{(i)}$
and the corresponding true label is $y^{(i)}$,
the *squared error* is given by:

$$l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2.$$
:eqlabel:`eq_mse`

The constant $\frac{1}{2}$ makes no real difference
but proves to be notationally convenient,
since it cancels out when we take the derivative of the loss.
Because the training dataset is given to us,
and thus is out of our control,
the empirical error is only a function of the model parameters.
In :numref:`fig_fit_linreg`, we visualize the fit of a linear regression model
in a problem with one-dimensional inputs.

![Fitting a linear regression model to one-dimensional data.](../img/fit-linreg.svg)
:label:`fig_fit_linreg`

Note that large differences between
estimates $\hat{y}^{(i)}$ and targets $y^{(i)}$
lead to even larger contributions to the loss,
due to its quadratic form
(this quadraticity can be a double-edge sword; while it encourages the model to avoid large errors
it can also lead to excessive sensitivity to anomalous data).
To measure the quality of a model on the entire dataset of $n$ examples,
we simply average (or equivalently, sum)
the losses on the training set:

$$L(\mathbf{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\mathbf{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)^2.$$

When training the model, we seek parameters ($\mathbf{w}^*, b^*$)
that minimize the total loss across all training examples:

$$\mathbf{w}^*, b^* = \operatorname*{argmin}_{\mathbf{w}, b}\  L(\mathbf{w}, b).$$

### Analytic Solution

Unlike most of the models that we will cover,
linear regression presents us with
a surprisingly easy optimization problem.
In particular, we can find the optimal parameters
(as assessed on the training data)
analytically by applying a simple formula as follows.
First, we can subsume the bias $b$ into the parameter $\mathbf{w}$
by appending a column to the design matrix consisting of all 1s.
Then our prediction problem is to minimize $\|\mathbf{y} - \mathbf{X}\mathbf{w}\|^2$.
As long as the design matrix $\mathbf{X}$ has full rank
(no feature is linearly dependent on the others),
then there will be just one critical point on the loss surface
and it corresponds to the minimum of the loss over the entire domain.
Taking the derivative of the loss with respect to $\mathbf{w}$
and setting it equal to zero yields:

$$\begin{aligned}
    \partial_{\mathbf{w}} \|\mathbf{y} - \mathbf{X}\mathbf{w}\|^2 =
    2 \mathbf{X}^\top (\mathbf{X} \mathbf{w} - \mathbf{y}) = 0
    \textrm{ and hence }
    \mathbf{X}^\top \mathbf{y} = \mathbf{X}^\top \mathbf{X} \mathbf{w}.
\end{aligned}$$

Solving for $\mathbf{w}$ provides us with the optimal solution
for the optimization problem.
Note that this solution 

$$\mathbf{w}^* = (\mathbf X^\top \mathbf X)^{-1}\mathbf X^\top \mathbf{y}$$

will only be unique
when the matrix $\mathbf X^\top \mathbf X$ is invertible,
i.e., when the columns of the design matrix
are linearly independent :cite:`Golub.Van-Loan.1996`.



While simple problems like linear regression
may admit analytic solutions,
you should not get used to such good fortune.
Although analytic solutions allow for nice mathematical analysis,
the requirement of an analytic solution is so restrictive
that it would exclude almost all exciting aspects of deep learning.

### Minibatch Stochastic Gradient Descent

Fortunately, even in cases where we cannot solve the models analytically,
we can still often train models effectively in practice.
Moreover, for many tasks, those hard-to-optimize models
turn out to be so much better that figuring out how to train them
ends up being well worth the trouble.

The key technique for optimizing nearly every deep learning model,
and which we will call upon throughout this book,
consists of iteratively reducing the error
by updating the parameters in the direction
that incrementally lowers the loss function.
This algorithm is called *gradient descent*.

The most naive application of gradient descent
consists of taking the derivative of the loss function,
which is an average of the losses computed
on every single example in the dataset.
In practice, this can be extremely slow:
we must pass over the entire dataset before making a single update,
even if the update steps might be very powerful :cite:`Liu.Nocedal.1989`.
Even worse, if there is a lot of redundancy in the training data,
the benefit of a full update is limited.

The other extreme is to consider only a single example at a time and to take
update steps based on one observation at a time.
The resulting algorithm, *stochastic gradient descent* (SGD)
can be an effective strategy :cite:`Bottou.2010`, even for large datasets.
Unfortunately, SGD has drawbacks, both computational and statistical.
One problem arises from the fact that processors are a lot faster
multiplying and adding numbers than they are
at moving data from main memory to processor cache.
It is up to an order of magnitude more efficient to
perform a matrix--vector multiplication
than a corresponding number of vector--vector operations.
This means that it can take a lot longer to process
one sample at a time compared to a full batch.
A second problem is that some of the layers,
such as batch normalization (to be described in :numref:`sec_batch_norm`),
only work well when we have access
to more than one observation at a time.

The solution to both problems is to pick an intermediate strategy:
rather than taking a full batch or only a single sample at a time,
we take a *minibatch* of observations :cite:`Li.Zhang.Chen.ea.2014`.
The specific choice of the size of the said minibatch depends on many factors,
such as the amount of memory, the number of accelerators,
the choice of layers, and the total dataset size.
Despite all that, a number between 32 and 256,
preferably a multiple of a large power of $2$, is a good start.
This leads us to *minibatch stochastic gradient descent*.

In its most basic form, in each iteration $t$,
we first randomly sample a minibatch $\mathcal{B}_t$
consisting of a fixed number $|\mathcal{B}|$ of training examples.
We then compute the derivative (gradient) of the average loss
on the minibatch with respect to the model parameters.
Finally, we multiply the gradient
by a predetermined small positive value $\eta$,
called the *learning rate*,
and subtract the resulting term from the current parameter values.
We can express the update as follows:

$$(\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b).$$

In summary, minibatch SGD proceeds as follows:
(i) initialize the values of the model parameters, typically at random;
(ii) iteratively sample random minibatches from the data,
updating the parameters in the direction of the negative gradient.
For quadratic losses and affine transformations,
this has a closed-form expansion:

$$\begin{aligned} \mathbf{w} & \leftarrow \mathbf{w} - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \partial_{\mathbf{w}} l^{(i)}(\mathbf{w}, b) && = \mathbf{w} - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \mathbf{x}^{(i)} \left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)\\ b &\leftarrow b -  \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \partial_b l^{(i)}(\mathbf{w}, b) &&  = b - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right). \end{aligned}$$
:eqlabel:`eq_linreg_batch_update`

Since we pick a minibatch $\mathcal{B}$
we need to normalize by its size $|\mathcal{B}|$.
Frequently minibatch size and learning rate are user-defined.
Such tunable parameters that are not updated
in the training loop are called *hyperparameters*.
They can be tuned automatically by a number of techniques, such as Bayesian optimization
:cite:`Frazier.2018`. In the end, the quality of the solution is
typically assessed on a separate *validation dataset* (or *validation set*).

After training for some predetermined number of iterations
(or until some other stopping criterion is met),
we record the estimated model parameters,
denoted $\hat{\mathbf{w}}, \hat{b}$.
Note that even if our function is truly linear and noiseless,
these parameters will not be the exact minimizers of the loss, nor even deterministic.
Although the algorithm converges slowly towards the minimizers
it typically will not find them exactly in a finite number of steps.
Moreover, the minibatches $\mathcal{B}$
used for updating the parameters are chosen at random.
This breaks determinism.

Linear regression happens to be a learning problem
with a global minimum
(whenever $\mathbf{X}$ is full rank, or equivalently,
whenever $\mathbf{X}^\top \mathbf{X}$ is invertible).
However, the loss surfaces for deep networks contain many saddle points and minima.
Fortunately, we typically do not care about finding
an exact set of parameters but merely any set of parameters
that leads to accurate predictions (and thus low loss).
In practice, deep learning practitioners
seldom struggle to find parameters
that minimize the loss *on training sets*
:cite:`Izmailov.Podoprikhin.Garipov.ea.2018,Frankle.Carbin.2018`.
The more formidable task is to find parameters
that lead to accurate predictions on previously unseen data,
a challenge called *generalization*.
We return to these topics throughout the book.

### Predictions

Given the model $\hat{\mathbf{w}}^\top \mathbf{x} + \hat{b}$,
we can now make *predictions* for a new example,
e.g., predicting the sales price of a previously unseen house
given its area $x_1$ and age $x_2$.
Deep learning practitioners have taken to calling the prediction phase *inference*
but this is a bit of a misnomer---*inference* refers broadly
to any conclusion reached on the basis of evidence,
including both the values of the parameters
and the likely label for an unseen instance.
If anything, in the statistics literature
*inference* more often denotes parameter inference
and this overloading of terminology creates unnecessary confusion
when deep learning practitioners talk to statisticians.
In the following we will stick to *prediction* whenever possible.



## Vectorization for Speed

When training our models, we typically want to process
whole minibatches of examples simultaneously.
Doing this efficiently requires that (**we**) (~~should~~)
(**vectorize the calculations and leverage
fast linear algebra libraries
rather than writing costly for-loops in Python.**)

To see why this matters so much,
let's (**consider two methods for adding vectors.**)
To start, we instantiate two 10,000-dimensional vectors
containing all 1s.
In the first method, we loop over the vectors with a Python for-loop.
In the second, we rely on a single call to `+`.

## 기본

*선형 회귀*는 회귀 문제를 해결하는 표준 도구 중 가장 간단하면서도 가장 널리 사용되는 방법입니다.19세기 초 :cite:`Legendre.1805,Gauss.1809`로 거슬러 올라가는 선형 회귀는 몇 가지 간단한 가정에서 시작됩니다.첫째, 특징 $\mathbf{x}$와 목표 $y$ 사이의 관계가
근사적으로 선형이라고 가정합니다.
즉, 조건부 평균 $E[Y \mid X=\mathbf{x}]$은 특징 $\mathbf{x}$의 가중 합으로 표현될 수 있습니다.이러한 설정은 목표 값이 관측치 잡음으로 인해
기대값에서 벗어날 수 있음을 허용합니다.다음으로, 이러한 잡음은 가우스 분포를 따르는
정상적으로 동작한다는 가정을 할 수 있습니다.일반적으로 $n$은 데이터세트의
예제 개수를 나타냅니다.위 첨자는 샘플과 타겟을 열거하고,
아래 첨자는 좌표를 색인합니다.더 구체적으로
$\mathbf{x}^{(i)}$는 $i^{\textrm{th}}$ 번째 샘플을 나타내고
$x_j^{(i)}$는 $j^{\textrm{th}}$ 번째 좌표를 나타냅니다.

### 모델
:label:`subsec_linear_model`

모든 솔루션의 핵심은
특징을 어떻게 변환하여 타겟의 추정값으로 사용할 수 있는지 설명하는 모델입니다.선형성 가정은
목표(가격)의 기댓값을
특성(면적과 연령)의 가중 합으로 표현할 수 있음을 의미합니다.

$$\textrm{price} = w_{\textrm{area}} \cdot \textrm{area} + w_{\textrm{age}} \cdot \textrm{age} + b.$$
:eqlabel:`eq_price-area`

여기서 $w_{\textrm{area}}$와 $w_{\textrm{age}}$를
*가중치*라고 하고, $b$를 *편향*(또는 *오프셋* 또는 *절편*)이라고 합니다.가중치는 각 특성이 예측에 미치는 영향을 결정합니다.편향은 모든 특성이 0일 때의 추정값을 결정합니다.새로 지어진 면적이 정확히 0인 주택은 볼 수 없겠지만,
편향은 여전히 필요합니다. 왜냐하면 편향은
특징의 모든 선형 함수를 표현할 수 있게 해 주기 때문입니다.
(원점을 통과하는 선으로 제한하는 것이 아니라).엄밀히 말하면, :eqref:`eq_price-area`는 입력 특징의 *아핀 변환*이며, 이는 가중 합을 통한 특징의 *선형 변환*과 추가된 편향을 통한 *변환*을 결합한 것으로 특징지어집니다.데이터 집합이 주어졌을 때, 우리의 목표는
평균적으로 모델의 예측이
데이터에서 관찰된 실제 가격에 최대한 가깝게 일치하도록 하는 가중치 $\mathbf{w}$와 편향 $b$를 선택하는 것입니다.특징이 몇 개뿐인 데이터 집합에 집중하는 것이 일반적인 분야에서는
:eqref:`eq_price-area`처럼
명시적으로 모델을 긴 형식으로 표현하는 것이 일반적입니다.머신 러닝에서는 일반적으로
고차원 데이터 세트를 사용하는데,
이 경우
간단한 선형 대수 표기법을 사용하는 것이 더 편리합니다.입력이 $d$개의 특성으로 구성될 때,
각 특성에 인덱스($1$과 $d$ 사이)를 할당하고
예측값 $\hat{y}$를 다음과 같이 표현할 수 있습니다.
(일반적으로 "모자" 기호는 추정값을 나타냄)

$$\hat{y} = w_1 x_1 + \cdots + w_d x_d + b.$$

모든 특성을 벡터 $\mathbf{x} \in \mathbb{R}^d$로
수집하고 모든 가중치를 벡터 $\mathbf{w} \in \mathbb{R}^d$로 수집하면
$\mathbf{w}$와 $\mathbf{x}$의 내적을 통해 모델을 간결하게 표현할 수 있습니다.

$$\hat{y} = \mathbf{w}^\top \mathbf{x} +b.$$
:eqlabel:`eq_linreg-y`

:eqref:`eq_linreg-y`에서 벡터 $\mathbf{x}$는
단일 예제의 특징에 해당합니다.종종 $n$개의 예제로 구성된 전체 데이터 세트의 특징을 *설계 행렬* $\mathbf{X} \in \mathbb{R}^{n \times d}$를 통해 참조하는 것이 편리합니다.여기서 $\mathbf{X}$는 각 예제에 대해 하나의 행과
각 특징에 대해 하나의 열을 포함합니다.특징 $\mathbf{X}$의 집합에 대해
예측 $\hat{\mathbf{y}} \in \mathbb{R}^n$은 다음과 같이 행렬-벡터 곱으로 표현할 수 있습니다.

$${\hat{\mathbf{y}}} = \mathbf{X} \mathbf{w} + b,$$
:eqlabel:`eq_linreg-y-vec`

여기서 브로드캐스팅(:numref:`subsec_broadcasting`)은 합산 중에 적용됩니다.학습 데이터 세트 $\mathbf{X}$의 특징과 그에 대응하는 (알려진) 레이블 $\mathbf{y}$가 주어졌을 때,
선형 회귀의 목표는
가중치 벡터 $\mathbf{w}$와 편향 항 $b$를 찾는 것입니다.
이 때, $\mathbf{X}$와 동일한 분포에서 샘플링된 새 데이터 샘플의 특징이 주어졌을 때,
새 샘플의 레이블이 (기대치에 따라) 가장 작은 오차로 예측됩니다.$\mathbf{x}$가 주어졌을 때 $y$를 예측하는 데 가장 적합한 모델이 선형이라고 믿더라도,
모든 $1 \leq i \leq n$에 대해
$y^{(i)}$가 $\mathbf{w}^\top \mathbf{x}^{(i)}+b$와 정확히 일치하는 $n$개의 샘플로 구성된 실제 데이터 세트를 찾을 수 있을 것이라고 기대하지는 않습니다.예를 들어, 특징 $\mathbf{X}$와 레이블 $\mathbf{y}$를 관찰하는 데 사용하는 도구가 무엇이든
소량의 측정 오류가 있을 수 있습니다.따라서 기본 관계가 선형이라고 확신하더라도
그러한 오류를 설명하기 위해 잡음 항을 통합합니다.최적의 *매개변수*
(또는 *모델 매개변수*) $\mathbf{w}$와 $b$를 찾기 전에
두 가지가 더 필요합니다.
(i) 주어진 모델의 품질에 대한 척도;
(ii) 모델의 품질을 개선하기 위한 업데이트 절차.

### 손실 함수
:label:`subsec_linear-regression-loss-function`

당연히 모델을 데이터에 적합시키려면
*적합성*(또는 *부적합성*)에 대한 어떤 척도에 동의해야 합니다.*손실 함수*는 목표의 *실제* 값과 *예측* 값 사이의 거리를 정량화합니다.손실은 일반적으로 음수가 아닌 값이며
값이 작을수록 더 좋으며
완벽한 예측에는 손실이 0입니다.
회귀 문제에서 가장 일반적인 손실 함수는 제곱 오차입니다.예제 $i$에 대한 예측값이 $\hat{y}^{(i)}$이고
해당 참 레이블이 $y^{(i)}$일 때,
*제곱 오차*는 다음과 같습니다.

$$l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2.$$
:eqlabel:`eq_mse`

상수 $\frac{1}{2}$는 실질적인 차이를 만들지 않지만
손실의 미분을 하면 상쇄되므로 표기상 편리합니다.훈련 데이터 세트는 우리에게 주어지고
따라서 우리가 통제할 수 없기 때문에
경험적 오차는 모델 매개변수의 함수일 뿐입니다.:numref:`fig_fit_linreg`에서 1차원 입력값을 갖는 문제에서 선형 회귀 모델의 적합도를 시각화합니다.![1차원 데이터에 선형 회귀 모델 적합하기.](../img/fit-linreg.svg)
:label:`fig_fit_linreg`

추정치 $\hat{y}^{(i)}$와 목표 $y^{(i)}$ 사이의 큰 차이는
이차 함수 형태 때문에 손실에 더 큰 영향을 미칩니다.
(이 이차 함수 형태는 양날의 검이 될 수 있습니다. 모델이 큰 오류를 피하도록 유도하는 반면,
이상 데이터에 대한 과도한 민감도로 이어질 수도 있습니다.)$n$개의 예제로 구성된 전체 데이터셋에서 모델의 품질을 측정하려면
단순히 훈련 세트의 손실을 평균(또는 동등하게 합산)합니다.

$$L(\mathbf{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\mathbf{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)^2.$$

모델을 훈련할 때 모든 훈련 예제에서 총 손실을 최소화하는 매개변수($\mathbf{w}^*, b^*$)를 찾습니다.

$$\mathbf{w}^*, b^* = \operatorname*{argmin}_{\mathbf{w},b}\ L(\mathbf{w}, b).$$

### 분석적 해결책

우리가 다룰 대부분의 모델과 달리
선형 회귀는
놀랍도록 쉬운 최적화 문제를 제시합니다.특히, 다음과 같은 간단한 공식을 적용하여
(훈련 데이터에서 평가된) 최적의 매개변수를 분석적으로 찾을 수 있습니다.먼저, 모든 값이 1인 설계 행렬에 열을 추가하여 편향 $b$를 매개변수 $\mathbf{w}$에 포함할 수 있습니다. 그러면 예측 문제는 $\|\mathbf{y} - \mathbf{X}\mathbf{w}\|^2$를 최소화하는 것입니다.설계 행렬 $\mathbf{X}$가 완전 순위를 갖는 한(어떤 특성도 다른 특성에 선형적으로 종속되지 않는 한),
손실 표면에는 하나의 임계점만 존재하며
이는 전체 도메인에서 손실의 최소값에 해당합니다.손실을 $\mathbf{w}$에 대해 미분하고
0으로 설정하면 다음과 같습니다.

$$\begin{aligned}
\partial_{\mathbf{w}} \|\mathbf{y} - \mathbf{X}\mathbf{w}\|^2 =
2 \mathbf{X}^\top (\mathbf{X} \mathbf{w} - \mathbf{y}) = 0
\textrm{ 따라서 }
\mathbf{X}^\top \mathbf{y} = \mathbf{X}^\top \mathbf{X} \mathbf{w}.\end{aligned}$$

$\mathbf{w}$에 대해 풀면 최적화 문제에 대한
최적의 해를 얻을 수 있습니다.이 해는

$$\mathbf{w}^* = (\mathbf X^\top \mathbf X)^{-1}\mathbf X^\top \mathbf{y}$$

유일한 해는
행렬 $\mathbf X^\top \mathbf X$가 가역적일 때, 즉 설계 행렬의 열이
선형적으로 독립일 때만 유일합니다.:cite:`Golub.Van-Loan.1996`

선형 회귀와 같은 간단한 문제에는
해석적 해가 허용될 수 있지만,
그런 행운에 익숙해져서는 안 됩니다.해석적 해는 훌륭한 수학적 분석을 가능하게 하지만,
해석적 해의 요구 조건이 너무 제한적이어서
딥러닝의 거의 모든 흥미로운 측면을 배제하게 됩니다.

### 미니배치 확률적 경사하강법

다행히 모델을 분석적으로 풀 수 없는 경우에도
실제로는 모델을 효과적으로 학습시킬 수 있는 경우가 많습니다.더욱이, 많은 작업에서 최적화하기 어려운 모델이 훨씬 더 나은 결과를 보여주기 때문에
이러한 모델을 학습시키는 방법을 알아내는 것이
결국 그만한 가치가 있는 것으로 나타납니다. 거의 모든 딥러닝 모델을 최적화하는 핵심 기법은
이 책 전체에서 다룰 것이며,
손실 함수를 점진적으로 낮추는 방향으로 매개변수를 업데이트하여
오차를 반복적으로 줄이는 것입니다.이 알고리즘을 *경사하강법*이라고 합니다. **경사하강법의 가장 단순한 응용은
손실 함수의 미분을 구하는 것입니다.**
미분은 데이터세트의 모든 단일 예제에서 계산된 손실의 평균입니다.실제로 이 작업은 매우 느릴 수 있습니다.
단일 업데이트를 하기 전에 전체 데이터 세트를 전달해야 합니다.
업데이트 단계가 매우 강력할 수 있더라도 말입니다. :cite:`Liu.Nocedal.1989`.
더 심각한 문제는 학습 데이터에 중복성이 많으면 전체 업데이트의 이점이 제한된다는 것입니다.또 다른 극단적인 방법은 한 번에 하나의 예만 고려하고 한 번에 하나의 관측치를 기반으로 업데이트 단계를 수행하는 것입니다. 결과적인 알고리즘인 *확률적 경사 하강법*(SGD)은 대규모 데이터 세트에서도 효과적인 전략이 될 수 있습니다.:cite:`Bottou.2010`
안타깝게도 SGD는 계산적, 통계적 측면에서 단점이 있습니다.한 가지 문제는 프로세서가 데이터를 메인 메모리에서 프로세서 캐시로 이동하는 것보다 숫자를 곱하고 더하는 속도가 훨씬 빠르다는 사실에서 발생합니다. 행렬-벡터 곱셈을 수행하는 것이 동일한 수의 벡터-벡터 연산을 수행하는 것보다 최대 10배 더 효율적입니다.즉, 전체 배치를 처리하는 것보다 한 번에 하나의 샘플을 처리하는 데 훨씬 더 오랜 시간이 걸릴 수 있습니다.두 번째 문제는 배치 정규화(sec_batch_norm에서 설명)와 같은 일부 계층이 한 번에 두 개 이상의 관측치에 접근할 수 있을 때만 제대로 작동한다는 것입니다. 두 문제 모두에 대한 해결책은 중간 전략을 선택하는 것입니다.
즉, 한 번에 전체 배치나 단일 샘플만 가져오는 대신, 관측치의 *미니 배치*를 가져옵니다. :cite:`Li.Zhang.Chen.ea.2014`
해당 미니 배치의 크기를 구체적으로 선택하는 것은 메모리 양, 가속기 수, 계층 선택, 전체 데이터 세트 크기 등 여러 요인에 따라 달라집니다.이러한 모든 사항에도 불구하고, 32에서 256 사이의 숫자, 바람직하게는 2의 큰 거듭제곱의 배수가 좋은 시작점입니다. 이를 통해 *미니 배치 확률적 경사 하강법*을 사용할 수 있습니다. 가장 기본적인 형태로, 각 반복 $t$에서
먼저 고정된 개수의 $|\mathcal{B}|$개의 학습 예제로 구성된 미니배치 $\mathcal{B}_t$를 무작위로 샘플링합니다.그런 다음 모델 매개변수에 대한 미니배치의 평균 손실의 미분(기울기)을 계산합니다. 마지막으로, 기울기에 미리 정해진 작은 양수 값 $\eta$(학습률이라고 함)를 곱하고
현재 매개변수 값에서 결과 항을 뺍니다.업데이트는 다음과 같이 표현할 수 있습니다.

$$(\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b).$$

요약하면, 미니배치 SGD는 다음과 같이 진행됩니다.
(i) 일반적으로 무작위로 모델 매개변수 값을 초기화합니다.(ii) 데이터에서 무작위 미니배치를 반복적으로 샘플링하여
음의 기울기 방향으로 매개변수를 업데이트합니다.이차 손실과 아핀 변환의 경우,
다음과 같은 닫힌 형태의 전개가 있습니다.

$$\begin{aligned} \mathbf{w} & \leftarrow \mathbf{w} - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \partial_{\mathbf{w}} l^{(i)}(\mathbf{w}, b) && = \mathbf{w} - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \mathbf{x}^{(i)} \left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)\\ b &\leftarrow b -\frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \partial_b l^{(i)}(\mathbf{w}, b) && = b - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}_t} \left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right).\end{aligned}$$
:eqlabel:`eq_linreg_batch_update`

미니 배치 $\mathcal{B}$를 선택했으므로
미니 배치의 크기 $|\mathcal{B}|$로 정규화해야 합니다.미니 배치 크기와 학습률은 사용자가 정의하는 경우가 많습니다.학습 루프에서 업데이트되지 않는 이러한 조정 가능한 매개변수를 *초매개변수*라고 합니다.베이지안 최적화와 같은 여러 기법을 통해 자동으로 조정할 수 있습니다.
:cite:`Frazier.2018`.결국, 솔루션의 품질은 일반적으로 별도의 *검증 데이터셋*(또는 *검증 세트*)에서 평가됩니다.미리 정해진 반복 횟수 동안 학습한 후(또는 다른 중단 기준이 충족될 때까지),
추정된 모델 매개변수를 기록하며, 이를 $\hat{\mathbf{w}}, \hat{b}$로 표시합니다.함수가 실제로 선형적이고 잡음이 없더라도
이러한 매개변수는 손실을 정확하게 최소화하는 매개변수가 아니며, 결정론적 매개변수도 아닙니다.알고리즘은 최소화하는 매개변수를 향해 천천히 수렴하지만, 일반적으로 유한한 단계 안에 정확하게 찾지는 못합니다.또한, 매개변수 업데이트에 사용되는 미니 배치 $\mathcal{B}$는 무작위로 선택됩니다.이것은 결정론을 깨뜨립니다.선형 회귀는 전역 최소값을 갖는 학습 문제입니다.
($\mathbf{X}$가 완전 랭크일 때, 또는 동등하게 $\mathbf{X}^\top \mathbf{X}$가 가역적일 때).그러나 심층 신경망의 손실 표면은 많은 안장점과 최소값을 포함합니다.다행히도 우리는 일반적으로 정확한 매개변수 집합을 찾는 데 관심이 없고, 정확한 예측(따라서 낮은 손실)으로 이어지는 매개변수 집합만 찾습니다.실제로 심층 학습 실무자는 *훈련 세트*에서 손실을 최소화하는 매개변수를 찾는 데 어려움을 겪는 경우가 거의 없습니다.
:cite:`Izmailov.Podoprikhin.Garipov.ea.2018,Frankle.Carbin.2018`.더 어려운 작업은 이전에 보지 못한 데이터에 대한 정확한 예측으로 이어지는 매개변수를 찾는 것입니다.
*일반화*라는 과제입니다.이 책 전반에 걸쳐 이 주제들을 다시 다룹니다.

### 예측

모델 $\hat{\mathbf{w}}^\top \mathbf{x} + \hat{b}$가 주어졌을 때,
이제 새로운 예시에 대해 *예측*을 할 수 있습니다.
예를 들어, 이전에 보지 못했던 주택의 면적 $x_1$과 연령 $x_2$을 고려하여 매매 가격을 예측하는 것입니다.딥러닝 실무자들은 예측 단계를 *추론*이라고 부르지만, 이는 다소 잘못된 표현입니다. *추론*은 매개변수 값과 보이지 않는 사례에 대한 가능성 있는 레이블을 포함하여
증거를 기반으로 도출된 모든 결론을 광범위하게 지칭합니다.통계학 문헌에서 *추론*은 매개변수 추론을 의미하는 경우가 더 많으며,
이러한 용어의 과다 사용은 딥러닝 실무자들이 통계학자들과 이야기할 때 불필요한 혼란을 야기합니다.다음에서는 가능하면 *예측*을 고수하겠습니다.## 속도를 위한 벡터화

모델을 학습할 때 일반적으로 전체 미니 배치 예제를 동시에 처리하려고 합니다.이를 효율적으로 수행하려면 (**우리**) (~~해야 합니다~~)
(**계산을 벡터화하고 Python에서 값비싼 for 루프를 작성하는 대신
빠른 선형 대수 라이브러리를 활용해야 합니다.**)

이것이 왜 중요한지 알아보기 위해
(**벡터를 더하는 두 가지 방법을 생각해 보겠습니다.**)
먼저, 모든 값이 1인 10,000차원 벡터 두 개를 인스턴스화합니다.첫 번째 방법에서는 Python for 루프를 사용하여 벡터를 반복합니다.두 번째 방법에서는 `+`를 한 번 호출합니다.


In [4]:
n = 10000
a = torch.ones(n)
b = torch.ones(n)

Now we can benchmark the workloads.
First, [**we add them, one coordinate at a time,
using a for-loop.**]

이제 워크로드를 벤치마킹할 수 있습니다.먼저, [**for 루프를 사용하여 한 번에 한 좌표씩 추가합니다.**]


In [5]:
c = torch.zeros(n)
t = time.time()
for i in range(n):
    c[i] = a[i] + b[i]
f'{time.time() - t:.5f} sec'

'0.06384 sec'

(**Alternatively, we rely on the reloaded `+` operator to compute the elementwise sum.**)

(**또는, 다시 로드된 `+` 연산자를 사용하여 요소별 합을 계산합니다.**)


In [7]:
t = time.time()
d = a + b
f'{time.time() - t:.5f} sec'

'0.01652 sec'

The second method is dramatically faster than the first.
Vectorizing code often yields order-of-magnitude speedups.
Moreover, we push more of the mathematics to the library
so we do not have to write as many calculations ourselves,
reducing the potential for errors and increasing portability of the code.

두 번째 방법은 첫 번째 방법보다 훨씬 빠릅니다. 코드를 벡터화하면 종종 수십 배의 속도 향상을 얻을 수 있습니다.더욱이, 더 많은 수학 연산을 라이브러리에 할당하여
직접 계산을 많이 작성할 필요가 없어 오류 발생 가능성을 줄이고 코드의 이식성을 높입니다.

## The Normal Distribution and Squared Loss
:label:`subsec_normal_distribution_and_squared_loss`

So far we have given a fairly functional motivation
of the squared loss objective:
the optimal parameters return the conditional expectation $E[Y\mid X]$
whenever the underlying pattern is truly linear,
and the loss assigns large penalties for outliers.
We can also provide a more formal motivation
for the squared loss objective
by making probabilistic assumptions
about the distribution of noise.

Linear regression was invented at the turn of the 19th century.
While it has long been debated whether Gauss or Legendre
first thought up the idea,
it was Gauss who also discovered the normal distribution
(also called the *Gaussian*).
It turns out that the normal distribution
and linear regression with squared loss
share a deeper connection than common parentage.

To begin, recall that a normal distribution
with mean $\mu$ and variance $\sigma^2$ (standard deviation $\sigma$)
is given as

$$p(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(-\frac{1}{2 \sigma^2} (x - \mu)^2\right).$$

Below [**we define a function to compute the normal distribution**].


## 정규 분포와 제곱 손실
:label:`subsec_normal_distribution_and_squared_loss`

지금까지 제곱 손실 목표에 대한 상당히 기능적인 동기를 제시했습니다.
최적 매개변수는 기본 패턴이 실제로 선형이고
손실이 이상치에 대해 큰 페널티를 부여할 때 조건부 기대값 $E[Y\mid X]$을 반환합니다. 또한, 노이즈 분포에 대한 확률론적 가정을 통해
제곱 손실 목표에 대한 보다 공식적인 동기를 제공할 수 있습니다.선형 회귀는 19세기 초에 발명되었습니다. 가우스와 르장드르 중 누가 먼저 이 아이디어를 생각해 냈는지에 대해서는 오랫동안 논쟁이 있었지만, 정규분포(*가우스*라고도 함)를 발견한 사람은 가우스였습니다. 정규분포와 제곱 손실을 고려한 선형 회귀는
공통된 기원보다 더 깊은 연관성을 가지고 있는 것으로 밝혀졌습니다. 먼저, 평균 $\mu$와 분산 $\sigma^2$(표준편차 $\sigma$)를 갖는 정규분포는 다음과 같이 주어집니다.

$$p(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(-\frac{1}{2 \sigma^2} (x - \mu)^2\right).$$

아래에서 [**정규분포를 계산하는 함수를 정의합니다**].


In [8]:
def normal(x, mu, sigma):
    p = 1 / math.sqrt(2 * math.pi * sigma**2)
    return p * np.exp(-0.5 * (x - mu)**2 / sigma**2)

We can now (**visualize the normal distributions**).

이제 우리는 (**정규 분포를 시각화**)할 수 있습니다.


In [1]:
# Use NumPy again for visualization
x = np.arange(-7, 7, 0.01)

# Mean and standard deviation pairs
params = [(0, 1), (0, 2), (3, 1)]
d2l.plot(x, [normal(x, mu, sigma) for mu, sigma in params], xlabel='x',
         ylabel='p(x)', figsize=(4.5, 2.5),
         legend=[f'mean {mu}, std {sigma}' for mu, sigma in params])

NameError: name 'np' is not defined

Note that changing the mean corresponds
to a shift along the $x$-axis,
and increasing the variance
spreads the distribution out,
lowering its peak.

One way to motivate linear regression with squared loss
is to assume that observations arise from noisy measurements,
where the noise $\epsilon$ follows the normal distribution 
$\mathcal{N}(0, \sigma^2)$:

$$y = \mathbf{w}^\top \mathbf{x} + b + \epsilon \textrm{ where } \epsilon \sim \mathcal{N}(0, \sigma^2).$$

Thus, we can now write out the *likelihood*
of seeing a particular $y$ for a given $\mathbf{x}$ via

$$P(y \mid \mathbf{x}) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(-\frac{1}{2 \sigma^2} (y - \mathbf{w}^\top \mathbf{x} - b)^2\right).$$

As such, the likelihood factorizes.
According to *the principle of maximum likelihood*,
the best values of parameters $\mathbf{w}$ and $b$ are those
that maximize the *likelihood* of the entire dataset:

$$P(\mathbf y \mid \mathbf X) = \prod_{i=1}^{n} p(y^{(i)} \mid \mathbf{x}^{(i)}).$$

The equality follows since all pairs $(\mathbf{x}^{(i)}, y^{(i)})$
were drawn independently of each other.
Estimators chosen according to the principle of maximum likelihood
are called *maximum likelihood estimators*.
While, maximizing the product of many exponential functions,
might look difficult,
we can simplify things significantly, without changing the objective,
by maximizing the logarithm of the likelihood instead.
For historical reasons, optimizations are more often expressed
as minimization rather than maximization.
So, without changing anything,
we can *minimize* the *negative log-likelihood*,
which we can express as follows:

$$-\log P(\mathbf y \mid \mathbf X) = \sum_{i=1}^n \frac{1}{2} \log(2 \pi \sigma^2) + \frac{1}{2 \sigma^2} \left(y^{(i)} - \mathbf{w}^\top \mathbf{x}^{(i)} - b\right)^2.$$

If we assume that $\sigma$ is fixed,
we can ignore the first term,
because it does not depend on $\mathbf{w}$ or $b$.
The second term is identical
to the squared error loss introduced earlier,
except for the multiplicative constant $\frac{1}{\sigma^2}$.
Fortunately, the solution does not depend on $\sigma$ either.
It follows that minimizing the mean squared error
is equivalent to the maximum likelihood estimation
of a linear model under the assumption of additive Gaussian noise.


평균을 변경하면
x축을 따라 이동하는 것과 같고,
분산을 증가시키면
분포가 퍼져서
최고점이 낮아집니다.
제곱 손실을 갖는 선형 회귀를 유도하는 한 가지 방법은 관측치가 잡음이 있는 측정값에서 발생한다고 가정하는 것입니다.
여기서 잡음 $\epsilon$은 정규 분포를 따릅니다.
$\mathcal{N}(0, \sigma^2)$:

$$y = \mathbf{w}^\top \mathbf{x} + b + \epsilon \textrm{ 여기서 } \epsilon \sim \mathcal{N}(0, \sigma^2).$$

따라서 이제 주어진 $\mathbf{x}$에 대해 특정 $y$를 볼 *가능성*을 다음과 같이 쓸 수 있습니다.

$$P(y \mid \mathbf{x}) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(-\frac{1}{2 \sigma^2} (y - \mathbf{w}^\top \mathbf{x}- b)^2\right).$$

따라서 우도 인수분해가 이루어집니다.*최대 우도의 원리*에 따르면,
모수 $\mathbf{w}$와 $b$의 최적 값은
전체 데이터 집합의 *우도*를 최대화하는 값입니다.

$$P(\mathbf y \mid \mathbf X) = \prod_{i=1}^{n} p(y^{(i)} \mid \mathbf{x}^{(i)}).$$

모든 쌍 $(\mathbf{x}^{(i)}, y^{(i)})$이 서로 독립적으로 도출되었으므로 등식이 성립합니다.최대 우도의 원리에 따라 선택된 추정량을 *최대 우도 추정량*이라고 합니다.여러 지수 함수의 곱을 최대화하는 것은 어려워 보일 수 있지만,
목표를 변경하지 않고도 우도의 로그를 최대화함으로써 상황을 크게 단순화할 수 있습니다.역사적인 이유로 최적화는
최대화보다는 최소화로 더 자주 표현됩니다.따라서 아무것도 변경하지 않고도
음의 로그 가능도*를 *최소화*할 수 있으며,
다음과 같이 표현할 수 있습니다.

$$-\log P(\mathbf y \mid \mathbf X) = \sum_{i=1}^n \frac{1}{2} \log(2 \pi \sigma^2) + \frac{1}{2 \sigma^2} \left(y^{(i)} - \mathbf{w}^\top \mathbf{x}^{(i)} - b\right)^2.$$

$\sigma$가 고정되어 있다고 가정하면
첫 번째 항은 무시할 수 있습니다.
첫 번째 항은 $\mathbf{w}$나 $b$에 의존하지 않기 때문입니다.두 번째 항은
앞서 소개한 제곱 오차 손실과 동일하지만,
곱셈 상수 $\frac{1}{\sigma^2}$만 다릅니다.다행히도 이 해는 $\sigma$에도 의존하지 않습니다.따라서 평균 제곱 오차를 최소화하는 것은
가산 가우시안 잡음을 가정할 때
선형 모델의 최대 우도 추정과 동일합니다.

## Linear Regression as a Neural Network

While linear models are not sufficiently rich
to express the many complicated networks
that we will introduce in this book,
(artificial) neural networks are rich enough
to subsume linear models as networks
in which every feature is represented by an input neuron,
all of which are connected directly to the output.

:numref:`fig_single_neuron` depicts
linear regression as a neural network.
The diagram highlights the connectivity pattern,
such as how each input is connected to the output,
but not the specific values taken by the weights or biases.

![Linear regression is a single-layer neural network.](../img/singleneuron.svg)
:label:`fig_single_neuron`

The inputs are $x_1, \ldots, x_d$.
We refer to $d$ as the *number of inputs*
or the *feature dimensionality* in the input layer.
The output of the network is $o_1$.
Because we are just trying to predict
a single numerical value,
we have only one output neuron.
Note that the input values are all *given*.
There is just a single *computed* neuron.
In summary, we can think of linear regression
as a single-layer fully connected neural network.
We will encounter networks
with far more layers
in later chapters.


## 신경망으로서의 선형 회귀

선형 모델은 이 책에서 소개할 여러 복잡한 네트워크를 표현할 만큼 충분히 풍부하지는 않지만
(인공) 신경망은
선형 모델을 네트워크로 포괄할 만큼 충분히 풍부합니다.
여기서 모든 특징은 입력 뉴런으로 표현되고,
모든 뉴런은 출력에 직접 연결됩니다.:numref:`fig_single_neuron`은
선형 회귀를 신경망으로 표현합니다.이 그림은 각 입력이 출력에 어떻게 연결되는지와 같은 연결 패턴을 강조하지만,
가중치나 편향이 취하는 구체적인 값은 강조하지 않습니다.![선형 회귀는 단일 계층 신경망입니다.](../img/singleneuron.svg)
:label:`fig_single_neuron`

입력은 $x_1, \ldots, x_d$입니다.$d$는 입력의 개수* 또는 입력 계층의 *특징 차원*을 나타냅니다.신경망의 출력은 $o_1$입니다.단일 숫자 값만 예측하려고 하므로
출력 뉴런은 하나뿐입니다.입력 값은 모두 *주어진* 값입니다.계산된 뉴런은 단 하나뿐입니다.요약하면, 선형 회귀는
단일 계층 완전 연결 신경망으로 생각할 수 있습니다.나중 장에서 훨씬 더 많은 계층을 가진 신경망을 살펴보겠습니다.


### Biology

Because linear regression predates computational neuroscience,
it might seem anachronistic to describe
linear regression in terms of neural networks.
Nonetheless, they were a natural place to start
when the cyberneticists and neurophysiologists
Warren McCulloch and Walter Pitts began to develop
models of artificial neurons.
Consider the cartoonish picture
of a biological neuron in :numref:`fig_Neuron`,
consisting of *dendrites* (input terminals),
the *nucleus* (CPU), the *axon* (output wire),
and the *axon terminals* (output terminals),
enabling connections to other neurons via *synapses*.

![The real neuron (source: "Anatomy and Physiology" by the US National Cancer Institute's Surveillance, Epidemiology and End Results (SEER) Program).](../img/neuron.svg)
:label:`fig_Neuron`

Information $x_i$ arriving from other neurons
(or environmental sensors) is received in the dendrites.
In particular, that information is weighted
by *synaptic weights* $w_i$,
determining the effect of the inputs,
e.g., activation or inhibition via the product $x_i w_i$.
The weighted inputs arriving from multiple sources
are aggregated in the nucleus
as a weighted sum $y = \sum_i x_i w_i + b$,
possibly subject to some nonlinear postprocessing via a function $\sigma(y)$.
This information is then sent via the axon to the axon terminals,
where it reaches its destination
(e.g., an actuator such as a muscle)
or it is fed into another neuron via its dendrites.

Certainly, the high-level idea that many such units
could be combined, provided they have the correct connectivity and learning algorithm,
to produce far more interesting and complex behavior
than any one neuron alone could express
arises from our study of real biological neural systems.
At the same time, most research in deep learning today
draws inspiration from a much wider source.
We invoke :citet:`Russell.Norvig.2016`
who pointed out that although airplanes might have been *inspired* by birds,
ornithology has not been the primary driver
of aeronautics innovation for some centuries.
Likewise, inspiration in deep learning these days
comes in equal or greater measure
from mathematics, linguistics, psychology,
statistics, computer science, and many other fields.

### 생물학

선형 회귀는 계산 신경과학보다 앞서 존재하기 때문에
선형 회귀를 신경망이라는 용어로 설명하는 것은 시대착오적으로 보일 수 있습니다.그럼에도 불구하고, 사이버네틱스와 신경생리학자인 워런 맥컬로치와 월터 피츠가 인공 뉴런 모델을 개발하기 시작했을 때, 신경망은 자연스럽게 시작점이 되었습니다.:numref:`fig_Neuron`의 생물학적 뉴런을 만화처럼 묘사한 그림을 생각해 보세요.
*수상돌기*(입력 단자), *핵*(CPU), *축삭*(출력 배선), 그리고 *축삭 말단*(출력 단자)으로 구성되어 있으며, *시냅스*를 통해 다른 뉴런과 연결됩니다.![실제 뉴런(출처: 미국 국립암연구소 감시, 역학 및 최종 결과(SEER) 프로그램의 "해부학 및 생리학").](../img/neuron.svg)
:label:`fig_Neuron`

다른 뉴런(또는 환경 센서)에서 수신된 정보 $x_i$는 수상돌기에서 수신됩니다.특히, 이 정보는 *시냅스 가중치* $w_i$에 의해 가중치가 부여되어
입력의 효과, 예를 들어 $x_i w_i$ 곱을 통한 활성화 또는 억제를 결정합니다.여러 소스에서 수신된 가중치가 부여된 입력은
핵에서 가중 합 $y = \sum_i x_i w_i + b$로 집계되며,
함수 $\sigma(y)$를 통한 비선형 후처리가 적용될 수 있습니다.이 정보는 축삭을 통해 축삭 말단으로 전송되고,
목적지(예: 근육과 같은 작동기)에 도달하거나

수상돌기를 통해 다른 뉴런으로 전달됩니다.
물론, 이러한 여러 단위들이 올바른 연결성과 학습 알고리즘만 있다면 결합되어 단일 뉴런만으로는 표현할 수 없는 훨씬 더 흥미롭고 복잡한 행동을 생성할 수 있다는 고차원적인 아이디어는 실제 생물학적 신경계 연구에서 비롯되었습니다.동시에 오늘날 딥 러닝 연구는 훨씬 더 광범위한 출처에서 영감을 얻고 있습니다.러셀 노비그(Russell.Norvig.2016)는 비행기가 새에게서 *영감*을 받았을지 모르지만, 조류학은 수 세기 동안 항공 혁신의 주요 동력이 아니었다고 지적했습니다.마찬가지로, 오늘날 딥 러닝의 영감은 수학, 언어학, 심리학, 통계학, 컴퓨터 과학 등 여러 분야에서 동등하거나 그 이상으로 많이 나옵니다.


## Summary

In this section, we introduced
traditional linear regression,
where the parameters of a linear function
are chosen to minimize squared loss on the training set.
We also motivated this choice of objective
both via some practical considerations
and through an interpretation
of linear regression as maximimum likelihood estimation
under an assumption of linearity and Gaussian noise.
After discussing both computational considerations
and connections to statistics,
we showed how such linear models could be expressed
as simple neural networks where the inputs
are directly wired to the output(s).
While we will soon move past linear models altogether,
they are sufficient to introduce most of the components
that all of our models require:
parametric forms, differentiable objectives,
optimization via minibatch stochastic gradient descent,
and ultimately, evaluation on previously unseen data.


## 요약

이 섹션에서는 전통적인 선형 회귀 분석을 소개했습니다.
여기서는 선형 함수의 매개변수가 훈련 세트의 제곱 손실을 최소화하도록 선택됩니다.또한 이러한 목표 선택은 몇 가지 실질적인 고려 사항과 선형 회귀를 선형성과 가우시안 잡음을 가정한 최대 가능도 추정으로 해석함으로써 이루어졌습니다.계산적 고려 사항과 통계와의 연관성을 논의한 후, 이러한 선형 모델을 입력이 출력에 직접 연결된 간단한 신경망으로 표현하는 방법을 보여주었습니다.곧 선형 모델을 완전히 벗어나겠지만, 모든 모델에 필요한 대부분의 구성 요소를 도입하기에 충분합니다. 즉, 매개변수 형태, 미분 가능한 목표, 미니배치 확률적 경사 하강법을 통한 최적화, 그리고 궁극적으로 이전에 본 적이 없는 데이터에 대한 평가입니다.


## Exercises
## 연습문제

1. Assume that we have some data $x_1, \ldots, x_n \in \mathbb{R}$. Our goal is to find a constant $b$ such that $\sum_i (x_i - b)^2$ is minimized.
    1. Find an analytic solution for the optimal value of $b$.
    1. How does this problem and its solution relate to the normal distribution?
    1. What if we change the loss from $\sum_i (x_i - b)^2$ to $\sum_i |x_i-b|$? Can you find the optimal solution for $b$?

1. 데이터 $x_1, \ldots, x_n \in \mathbb{R}$이 있다고 가정합니다.목표는 $\sum_i (x_i - b)^2$가 최소화되는 상수 $b$를 찾는 것입니다.
   1. $b$의 최적값에 대한 해석적 해를 구하시오.
   1. 이 문제와 그 해는 정규분포와 어떤 관련이 있는가?
   1. 손실을 $\sum_i (x_i - b)^2$에서 $\sum_i |x_i-b|$로 바꾸면 어떻게 될까?$b$의 최적해를 구할 수 있는가?

1. Prove that the affine functions that can be expressed by $\mathbf{x}^\top \mathbf{w} + b$ are equivalent to linear functions on $(\mathbf{x}, 1)$.
1. $\mathbf{x}^\top \mathbf{w} + b$로 표현될 수 있는 아핀 함수가 $(\mathbf{x}, 1)$에서의 선형 함수와 동치임을 증명하시오.

1. Assume that you want to find quadratic functions of $\mathbf{x}$, i.e., $f(\mathbf{x}) = b + \sum_i w_i x_i + \sum_{j \leq i} w_{ij} x_{i} x_{j}$. How would you formulate this in a deep network?
1. $\mathbf{x}$의 이차 함수, 즉 $f(\mathbf{x}) = b + \sum_i w_i x_i + \sum_{j \leq i} w_{ij} x_{i} x_{j}$를 구한다고 가정하자. 심층 신경망에서 이를 어떻게 공식화할 수 있을까요?

1. Recall that one of the conditions for the linear regression problem to be solvable was that the design matrix $\mathbf{X}^\top \mathbf{X}$ has full rank.
    1. What happens if this is not the case?
    1. How could you fix it? What happens if you add a small amount of coordinate-wise independent Gaussian noise to all entries of $\mathbf{X}$?
    1. What is the expected value of the design matrix $\mathbf{X}^\top \mathbf{X}$ in this case?
    1. What happens with stochastic gradient descent when $\mathbf{X}^\top \mathbf{X}$ does not have full rank?
1. 선형 회귀 문제가 해결 가능하기 위한 조건 중 하나는 설계 행렬 $\mathbf{X}^\top \mathbf{X}$가 완전 랭크를 가져야 한다는 것이었습니다.
   1. 그렇지 않다면 어떻게 될까요?
   1. 어떻게 해결할 수 있을까요?$\mathbf{X}$의 모든 항목에 좌표 방향으로 독립적인 가우시안 잡음을 소량 추가하면 어떻게 될까요?
   1. 이 경우 설계 행렬 $\mathbf{X}^\top \mathbf{X}$의 기댓값은 얼마일까요?
   1. $\mathbf{X}^\top \mathbf{X}$가 완전 랭크를 갖지 않을 때 확률적 경사 하강법은 어떻게 될까요?


1. Assume that the noise model governing the additive noise $\epsilon$ is the exponential distribution. That is, $p(\epsilon) = \frac{1}{2} \exp(-|\epsilon|)$.
    1. Write out the negative log-likelihood of the data under the model $-\log P(\mathbf y \mid \mathbf X)$.
    1. Can you find a closed form solution?
    1. Suggest a minibatch stochastic gradient descent algorithm to solve this problem. What could possibly go wrong (hint: what happens near the stationary point as we keep on updating the parameters)? Can you fix this?

1. 가산 잡음 $\epsilon$을 지배하는 잡음 모델이 지수 분포라고 가정합니다.즉, $p(\epsilon) = \frac{1}{2} \exp(-|\epsilon|)$입니다.
   1. $-\log P(\mathbf y \mid \mathbf X)$ 모델에 따른 데이터의 음의 로그 우도를 구하시오.
   1. 닫힌 형태의 해를 구할 수 있습니까?
   1. 이 문제를 해결하기 위한 미니배치 확률적 경사 하강법 알고리즘을 제안하시오. 무엇이 잘못될 수 있습니까?(힌트: 매개변수를 계속 업데이트할 때 정상점 근처에서 어떤 일이 발생합니까?) 이 문제를 해결할 수 있습니까?

1. Assume that we want to design a neural network with two layers by composing two linear layers. That is, the output of the first layer becomes the input of the second layer. Why would such a naive composition not work?
1. 두 개의 선형 층을 구성하여 두 개의 층을 가진 신경망을 설계한다고 가정해 보자.즉, 첫 번째 층의 출력이 두 번째 층의 입력이 된다.왜 이렇게 단순한 구성이 작동하지 않을까?


1. What happens if you want to use regression for realistic price estimation of houses or stock prices?
    1. Show that the additive Gaussian noise assumption is not appropriate. Hint: can we have negative prices? What about fluctuations?
    1. Why would regression to the logarithm of the price be much better, i.e., $y = \log \textrm{price}$?
    1. What do you need to worry about when dealing with pennystock, i.e., stock with very low prices? Hint: can you trade at all possible prices? Why is this a bigger problem for cheap stock? For more information review the celebrated Black--Scholes model for option pricing :cite:`Black.Scholes.1973`.

1. 주택이나 주가의 현실적인 가격 추정을 위해 회귀를 사용하려면 어떻게 해야 합니까?
   1. 가산 가우시안 잡음 가정이 적절하지 않음을 보여라.힌트: 음수 가격이 있을 수 있나요?가격 변동은 어떨까요?
   1. 가격의 로그로 회귀하는 것이 훨씬 더 나은 이유는 무엇인가요? 즉, $y = \log \textrm{price}$?
   1. 페니스톡(pennystock), 즉 매우 낮은 가격의 주식을 다룰 때 무엇을 고려해야 하나요?힌트: 가능한 모든 가격에서 거래할 수 있나요?저렴한 주식에서 이것이 더 큰 문제인 이유는 무엇인가요?더 자세한 내용은 유명한 옵션 가격 결정 모델인 Black-Scholes 모델을 참조하세요. :cite:`Black.Scholes.1973`.

1. Suppose we want to use regression to estimate the *number* of apples sold in a grocery store.
    1. What are the problems with a Gaussian additive noise model? Hint: you are selling apples, not oil.
    1. The [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) captures distributions over counts. It is given by $p(k \mid \lambda) = \lambda^k e^{-\lambda}/k!$. Here $\lambda$ is the rate function and $k$ is the number of events you see. Prove that $\lambda$ is the expected value of counts $k$.
    1. Design a loss function associated with the Poisson distribution.
    1. Design a loss function for estimating $\log \lambda$ instead.

1. 회귀를 사용하여 식료품점에서 판매되는 사과의 *개수*를 추정한다고 가정해 보겠습니다.
   1. 가우시안 가산 잡음 모델의 문제점은 무엇인가요?힌트: 당신은 기름이 아니라 사과를 판매하고 있습니다.
   1. [푸아송 분포](https://en.wikipedia.org/wiki/Poisson_distribution)는 개수에 따른 분포를 포착합니다.$p(k \mid \lambda) = \lambda^k e^{-\lambda}/k!$로 주어집니다.여기서 $\lambda$는 속도 함수이고 $k$는 보이는 사건의 개수입니다.$\lamda$가 개수 $k$의 기댓값임을 증명하세요.
   1. 포아송 분포와 관련된 손실 함수를 설계하세요.
   1. 대신 $\log \lambda$를 추정하는 손실 함수를 설계하세요.



[Discussions](https://discuss.d2l.ai/t/258)

[토론](https://discuss.d2l.ai/t/258)
